In [2]:
import tensorflow as tf 
import numpy as np 


In [11]:
class conv_layer(tf.keras.layers.Layer):
    def __init__(self , units  , kernal_size = 4, activation =None , pad = 0  , strides = 1) : 
        super(conv_layer,self).__init__()
        self.units = units 
        self.activation = tf.keras.activations.get(activation)
        self.padding = pad
        self.filter  = kernal_size 
        self.strides = strides

    # we will initalize the weights and biases
    def build(self , input_dims): 
        w_init =  tf.random_normal_initializer()
        # we initalize the weights with shape(kernal_size , kernal_size , number of input channels , number of units / new channels)
        self.weight = tf.Variable(w_init(shape =(self.filter , self.filter , input_dims[-1] , self.units)), dtype ='float32' , trainable =True)
        b_init = tf.zeros_initializer()
        self.bias = tf.Variable(b_init(shape = (1,1,1,self.units)) , dtype ='float32' , trainable =True)

    # to pad the input     
    def zero_pad(self , X, pad):

        X_pad = np.pad(X , ((0,0 ) , (pad , pad ) , (pad, pad ) , (0,0)) , mode='constant' , constant_values =(0,0)) 
        return X_pad

    # conv_window to multiply each slice of the image by the kernal weight and add the bais
    def conv_window(self, a_slice_prev, W, b):
        s = np.multiply(a_slice_prev,W)
        Z = np.sum(s)
        b = np.squeeze(b)
        Z = Z + b
        return Z 


    def fit(self , input):
        # we unpack the shape of the input 
        m, h_prev , w_prev , c_prev = input.shape
        pad = self.padding
        
        # calculate the new dims of the output 
        n_H = int(((h_prev - self.filter + 2 * pad) / self.strides) + 1)
        n_W = int(((w_prev - self.filter + 2 * pad) / self.strides) + 1)
        # init the output with zeros 
        out  = np.zeros((m, n_H, n_W, self.units))

        if pad != 0  :
             A_prev_pad = self.zero_pad(input, int(pad))
        A_prev_pad = input

        for i in range(m):  # looping throw the examples              
          a_prev_pad = A_prev_pad[i]         
          for h in range(n_H):  # looping throw the height dim          

              vert_start = self.strides * h  # calculate the vertical steps we will take to take a slice from the image 
              vert_end = vert_start  + self.filter
              for w in range(n_W):       
                  
                  horiz_start = self.strides * w  # calculate the horizontal steps we will take to take a slice from the image
                  horiz_end = horiz_start + self.filter
                   
                  for c in range(self.units):  # looping throw the channels 
                      
                      a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:] # take the slice which will be multipled by the kernal
      
                      weights = self.weight[:, :, :, c]
                      biases  = self.bias[:, :, :, c]
                      out[i, h, w, c] = self.conv_window(a_slice_prev, weights, biases) 
        return  self.activation(out)  # return the output of the activation 

In [4]:
# load the mnist to test the layer
(train_data , train_labels) , (test_data , test_lables) = tf.keras.datasets.fashion_mnist.load_data() 
# add new axis to ba able to unpack the dims inside the function 
train_data = train_data[:,:,: , np.newaxis]
train_data = train_data / 255

In [12]:
# just simple model to test the layer
model = tf.keras.models.Sequential([
          conv_layer(32 , activation='relu') , 
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(10 , activation ='softmax')
])

In [15]:
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics =['accuracy'])

In [ ]:
model.fit(train_data , train_labels, epochs=10)